In [1]:
%pip install sklearn
%pip install tensorflow tensorflow.keras

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Owner\.pyenv\pyenv-win\versions\3.8.10\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow.keras (from versions: none)
ERROR: No matching distribution found for tensorflow.keras
You should consider upgrading via the 'c:\Users\Owner\.pyenv\pyenv-win\versions\3.8.10\python.exe -m pip install --upgrade pip' command.


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

import csv
import cv2
import numpy as np
import decord
import torch
import os, time
import pandas as pd
import random

from gluoncv.torch.utils.model_utils import download
from gluoncv.torch.data.transforms.videotransforms import video_transforms, volume_transforms
from gluoncv.torch.engine.config import get_cfg_defaults
from gluoncv.torch.model_zoo import get_model
from tqdm.notebook import tqdm

## Train Class Prediction Model

In [ ]:
# function to load frames
def load_frames(frame_paths):
    frames = []
    for path in frame_paths:
        frame = cv2.imread(path)  # Load frame using OpenCV 
        frame = cv2.resize(frame, (224, 224))  # resizing to (224, 224)
        frames.append(frame)
    return np.array(frames)

# data preprocessing function
def preprocess_data(df):
    frames = load_frames(df['frames'])
    # Convert class IDs to categorical labels if needed
    labels = df['class_id']
    return frames, labels

In [ ]:
df = pd.read_csv("your_csv_file.csv")

# Preprocess data
X_train, y_train = preprocess_data(df)

# Concatenate frames and weights (assuming weights are random for now)
weights = np.random.rand(len(X_train), 1)
X_train_concat = np.concatenate([X_train.reshape(len(X_train), -1), weights], axis=1)

# Define model
model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train_concat.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax'),  # Assuming 10 action classes
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train_concat, y_train, epochs=10, batch_size=32)

model.save("G_Model.h5")

## Predict Frame Importance

In [5]:
map_path = 'action_map.csv'

action_dict = {}

with open(map_path, 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        id_value = int(row['id'])
        name_value = row['name']
        
        action_dict[id_value] = name_value

In [6]:
# Define a function to create the linear regression model
def create_linear_regression_model(num_frames):
    model = LinearRegression()
    return model

In [24]:
# Define a function to compute the predicted action class and confidence based on the frames and weights
def compute_predictions_linear_regression(target_data, num_classes, frames, weights, model):

    # Combine frames and weights into a single input matrix
    weights_reshaped = np.expand_dims(np.expand_dims(np.expand_dims(weights, axis=1), axis=2), axis=3)
    weights_reshaped = np.broadcast_to(weights_reshaped, frames.shape)
    input_data = np.concatenate((frames, weights_reshaped), axis=1)

    # Fit the linear regression model
    model.fit(input_data, target_data)  # target data is [class, confidence]

    # Predict the action class and confidence
    predicted_action_class = model.predict(input_data[:, :num_classes])
    predicted_confidence = model.predict(input_data[:, num_classes:])

    return predicted_action_class, predicted_confidence

In [14]:
def compute_loss(frames, weights, target_class, target_confidence, num_classes, model):
    # Compute the predicted action class and confidence based on the weights and frames
    predicted_class, predicted_confidence = compute_predictions_linear_regression([target_class, target_confidence], num_classes, frames, weights, model)

    # Compute the loss based on the predicted and target values
    loss = target_confidence - predicted_confidence
    if predicted_class != target_class:
        loss *= 100

    return loss

In [15]:
def optimize_on_video(frames, target_class, target_confidence, num_classes, model):

    # Define the initial weights for frames
    initial_weights = [1] * len(frames)

    # Define the optimization algorithm
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Perform optimization
    weights = tf.Variable(initial_weights, trainable=True)

    for step in range(1000):
        with tf.GradientTape() as tape:
            loss = compute_loss(frames, weights, target_class, target_confidence, num_classes, model)

        gradients = tape.gradient(loss, weights)
        optimizer.apply_gradients([(gradients, weights)])

        # Optional: Print the loss for monitoring
        if step % 100 == 0:
            print("Step:", step, "Loss:", loss.numpy())

    # Retrieve the optimized weights
    optimized_weights = weights.numpy()

    return optimized_weights

In [10]:
def get_frames(file_path, video_length):
    frame_id_list = [i for i in range(0, video_length, 2)]
    vr = decord.VideoReader(file_path)
    video_data = vr.get_batch(frame_id_list).asnumpy()
    crop_size = 224
    short_side_size = 256
    transform_fn = video_transforms.Compose([video_transforms.Resize(short_side_size, interpolation='bilinear'),
                                            video_transforms.CenterCrop(size=(crop_size, crop_size)),
                                            volume_transforms.ClipToTensor(),
                                            video_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    clip_input = transform_fn(video_data)

    return clip_input 

In [11]:
def f(video_input, model):
    with torch.no_grad():
            pred = model(torch.unsqueeze(video_input, dim=0)).numpy()
    probs = torch.nn.functional.softmax(torch.tensor(pred), dim=1).numpy()
    top_class = np.argmax(probs)
    confidence = np.max(probs) - np.min(probs)

    return top_class, confidence

In [26]:
num_classes = 5
num_videos = 5
video_length = 200 # ???????

frames_and_weights = [[]]

# Get 5 Random videos from collection
dataset_path = 'datasets\\kinetics400_5per\\train'

subfolders = sorted([subfolder for subfolder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, subfolder))])
random.seed(2)
random_subfolders = random.sample(subfolders, num_classes)
subfolder = random_subfolders[0]

config_file = './i3d_resnet50_v1_kinetics400.yaml'
cfg = get_cfg_defaults()
cfg.merge_from_file(config_file)
f_model = get_model(cfg)
f_model.eval()

g_model = load_model("G_Model.h5")

try:
    # Iterate over videos
    for index, subfolder in enumerate(random_subfolders):
        subfolder_path = os.path.join(dataset_path, subfolder)
        files = os.listdir(subfolder_path)
        clean_files = []
        for filename in files:
            if filename.endswith(".mp4"):
                clean_files.append(filename)
        random_files = random.sample(clean_files, num_videos)
        random_file = random_files[0]
        for random_file in random_files:
            file_path = os.path.join(subfolder_path, random_file)
            class_id = -1
            for key, value in action_dict.items():
                if value == subfolder:
                    class_id = key
                    break
            
            # get weights for video
            frames = get_frames(file_path, video_length)
            target_class, target_confidence = f(frames, f_model)
            optimized_frame_weights = optimize_on_video(frames, target_class, target_confidence, num_classes, g_model)

            # store frames and their weights
            frames_and_weights += [[frame, weight] for frame, weight in zip(frames, optimized_frame_weights)]
                
except Exception as e:
    print(e)

Found array with dim 4. LinearRegression expected <= 2.
